In [6]:
import pandas as pd
import numpy as np
import config
import time

In [3]:
def timestamp_datetime(value):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(value))

def time_feat(df,featList,featName):
    '''scaler = StandardScaler()
    tmp = df.groupby(featList).size().reset_index().rename(columns={0:featName})
    tmp[featName] = scaler.fit_transform(tmp[featName].values.reshape(-1,1))
    df = df.merge(tmp,'left',on=featList)'''
    df[featName] = df.groupby(featList)['context_timestamp'].rank(method='first')   
    return df

def process(df):
    df['time'] = df.context_timestamp.apply(timestamp_datetime)
    df['day'] = df.time.apply(lambda x: int(x[8:10]))
    df['hour'] = df.time.apply(lambda x: int(x[11:13]))
    '''for lst in timeFeatList:
        df = time_feat(df,lst,'_'.join(lst))'''
    df['item_property_list_clean'] = df['item_property_list'].apply(lambda x:';'.join(sorted(set(str(x).split(';')))))
    for i in range(3):
        df['item_category_list_bin%d'%i] = df['item_category_list'].apply(lambda x: x.split(';')[i] if len(x.split(';'))>i else -1)
    df["missing_feat"] = np.sum((df == -1).values, axis=1)
    return df

In [45]:
def focus_one_record(df,key_var=[]):
    nameBase = '_'.join(key_var)
    ###当天前后的数据情况
    df[nameBase+'_before_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='min') - 1
    df[nameBase+'_after_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='min',ascending=False)- 1
    df[nameBase+'_sametime_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='max') - df.groupby(key_var+['day'])['context_timestamp'].rank(method='min')+1
    df = df.merge(df.groupby(key_var+['day'],as_index=False)['context_timestamp'].agg({nameBase+'_day_cnt':'count'}),'inner',key_var+['day'])
    for feat in ['_before_cnt','_after_cnt','_sametime_cnt']:
        df[nameBase+feat+'_ratio'] = df[nameBase+feat]*1.0/df[nameBase+'_day_cnt']
        
    ###前一天购买/浏览的数量
    '''dfTmp =  df.groupby(key_var+['day'],as_index=False)['is_trade'].agg({nameBase+'_preday_trade_cnt':'sum',nameBase+'_preday_cnt':'count'})
    dfTmp['day'] = dfTmp['day']+1
    df = df.merge(dfTmp,'left',key_var+['day'])
    df[nameBase+'_preday_trade_ratio'] = df[nameBase+'_preday_trade_cnt']*1.0/df[nameBase+'_preday_cnt']
    for feat in ['_preday_trade_cnt','_preday_cnt','_preday_trade_ratio']:
        df[nameBase+feat].fillna(0,inplace=True)'''
        
    ###广告展示上下间隔
    dfTmp = df[[nameBase+'_before_cnt',nameBase+'_after_cnt',nameBase+'_sametime_cnt','time']+key_var+['day']]
    dfTmp.rename(columns={'time':'new_time'},inplace=True)
    dfTmp['next_record'] = dfTmp[nameBase+'_before_cnt'] + dfTmp[nameBase+'_sametime_cnt'] + 1
    dfTmp['last_record'] = dfTmp[nameBase+'_after_cnt'] + dfTmp[nameBase+'_sametime_cnt'] + 1
    df = df.merge(dfTmp[key_var+['day','next_record','new_time']],'left',left_on = key_var+['day',nameBase+'_before_cnt'],right_on = key_var+['day','next_record'])
    df['next_time_dur'] = (pd.to_datetime(df['time'])-pd.to_datetime(df['new_time'])).seconds    
    del df['new_time']
    
    df = df.merge(dfTmp[key_var+['day','last_record','new_time']],'left',left_on = key_var+['day',nameBase+'_after_cnt'],right_on = key_var+['day','last_record'])
    df['last_time_dur'] = (pd.to_datetime(df['new_time'])-pd.to_datetime(df['time'])).seconds  
    del df['new_time']
    return df    
    

In [39]:
dfTrain = pd.read_table(config.TRAIN_FILE,sep=' ')
dfTrain.drop_duplicates(inplace=True)
dfTrain.reset_index(inplace=True,drop =True)
dfTest = pd.read_table(config.TEST_FILE,sep=' ')

dfTrain = process(dfTrain)
dfTest = process(dfTest)

In [40]:
dfAll = pd.concat([dfTrain,dfTest])
dfAll.reset_index(inplace=True,drop=True)
dfAll.shape

(496482, 35)

In [46]:
focus_one_record(dfAll,['user_id'])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view

AttributeError: 'Series' object has no attribute 'seconds'

In [24]:
df = dfAll
key_var = ['user_id']

In [51]:
pd.to_datetime(dfAll['time'])

AttributeError: 'Series' object has no attribute 'seconds'

In [12]:
nameBase = '_'.join(key_var)
###当天前后的数据情况
df[nameBase+'_before_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='min')

KeyboardInterrupt: 

In [25]:
df.groupby('user_id')['context_timestamp'].rank(method='first')

0          1.0
1          2.0
2          1.0
3          1.0
4          1.0
5          2.0
6         20.0
7          1.0
8          1.0
9          4.0
10         1.0
11         1.0
12         1.0
13         3.0
14         1.0
15         1.0
16         1.0
17         1.0
18         4.0
19         1.0
20         3.0
21         1.0
22         1.0
23         1.0
24         1.0
25         1.0
26         1.0
27         1.0
28         1.0
29         3.0
          ... 
496452     1.0
496453     2.0
496454     4.0
496455     1.0
496456     1.0
496457     3.0
496458     1.0
496459     1.0
496460     1.0
496461    10.0
496462     2.0
496463     1.0
496464     3.0
496465     2.0
496466     2.0
496467     1.0
496468     1.0
496469     6.0
496470     1.0
496471     1.0
496472     1.0
496473     2.0
496474     1.0
496475     1.0
496476     1.0
496477     2.0
496478     1.0
496479     7.0
496480     2.0
496481     1.0
Name: context_timestamp, Length: 496482, dtype: float64

In [26]:
df.groupby('user_id')['context_timestamp'].rank(method='min')

0          1.0
1          3.0
2          1.0
3          1.0
4          1.5
5          2.0
6         20.5
7          1.0
8          1.0
9          4.0
10         1.0
11         1.0
12         1.0
13         3.0
14         1.0
15         1.0
16         1.5
17         1.0
18         4.0
19         1.5
20         3.0
21         2.0
22         1.0
23         1.0
24         1.0
25         1.0
26         1.0
27         1.5
28         1.0
29         3.0
          ... 
496452     1.0
496453     2.0
496454     4.0
496455     1.0
496456     1.0
496457     3.0
496458     1.0
496459     1.0
496460     1.0
496461    10.0
496462     2.0
496463     1.0
496464     3.0
496465     2.0
496466     2.0
496467     1.0
496468     1.0
496469     6.0
496470     1.0
496471     1.0
496472     1.0
496473     2.0
496474     1.0
496475     1.0
496476     1.0
496477     2.0
496478     1.0
496479     7.0
496480     2.0
496481     1.0
Name: context_timestamp, Length: 496482, dtype: float64

In [30]:
df.groupby(key_var+['day'])['context_timestamp'].count()

user_id              day
24779788309075       19     3
36134987234568       22     1
59341486148291       22     1
179317972644611      22     7
                     23     1
179451404933115      25     3
212049717332865      21     1
217191929432135      22     6
218627666998650      18     1
                     25     1
225768902336823      18     1
300164043453670      22     1
317896740941222      21     1
343687024536467      23     1
343846706165657      23     1
471191193416668      24     1
506604787274365      23     2
510416141439025      19     1
523117691035117      24     1
547560177346870      21     1
601357023584293      18     1
635591985978049      21     1
636099518731252      21     1
643902588708579      20     7
654859444444842      21     1
735230298329321      20     3
                     21     1
740725272974461      24     1
804651294451073      18     1
808429707297646      20     1
                           ..
9222418414773457583  23     1
922247656540339

In [15]:
key_var+['day']

['user_id', 'day']